In [459]:
# 학습 데이터 shuffle
# import random
# with open('./train_hate_dataset_v1.txt', 'r', encoding='utf-8') as f:
#     file = f.read()
# files = file.split('\n')
# with open('./test_hate_dataset_v1.txt', 'r', encoding='utf-8') as f:
#     file2 = f.read()
# files2 = file2.split('\n')
# files.extend(files2)
# random.shuffle(files)
# sentences = [x.split('\t') for x in files]
# len_sentences = len(sentences)
# splited_len = round(len_sentences*0.9)
# train_sentences = sentences[:splited_len]
# test_sentences = sentences[splited_len:]
# train_sentences = pd.DataFrame(train_sentences)
# test_sentences = pd.DataFrame(test_sentences)
# train_sentences.to_csv('./train_hate_dataset_v2.txt', sep='\t', index=None, header=None)
# test_sentences.to_csv('./test_hate_dataset_v2.txt', sep='\t', index=None, header=None)

In [8]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import jamotools #자모 단위 토큰화
import re
import gluonnlp as nlp
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer

class CharDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, mode):
        self.sentences = [line[sent_idx] for line in dataset]
        self.labels = [int(line[label_idx]) for line in dataset]
        self.korean = re.compile('[^1!ㄱ-ㅣ가-힣]+')
        self.mode = mode
        tok_path = '/home/yeonsik/kobert/kobert_news_wiki_ko_cased-1087f8699e.spiece'
        self.sp = SentencepieceTokenizer(tok_path)
        self.vocab = self.make_vocab()
        self.vocab_size = len(self.vocab)
        self.q3 = self.get_q3()
        self.char2idx = {u:i for i, u in enumerate(self.vocab)}
        self.idx2char = {i:u for i, u in enumerate(self.vocab)}
        self.max_len = self.find_max_len()
        
    def __getitem__(self, i):
        return (self.preprocess_sentence(self.sentences[i]), torch.tensor(self.labels[i]).to(torch.float32))
    
    def __len__(self):
        return len(self.labels)
    
    def make_vocab(self):
        vocab = ''
        for sentence in self.sentences:
            vocab+=' '+sentence
        vocab = self.make_token(vocab)
        vocab = set(vocab)
        vocab = sorted(vocab)
        vocab.append('<UNK>') #######
        vocab.append('<PAD>')
        return vocab
    
    def make_token(self, sentence):
        if self.mode == 'jamo':
            chars = self.korean.sub('', jamotools.split_syllables(sentence))
            return list(chars)
        elif self.mode == 'char':
            chars = self.korean.sub('', sentence)
            return list(chars)
        elif self.mode == 'sentencepiece':
            return self.sp(sentence)
        
    def preprocess_sentence(self, sentence):
        chars = self.make_token(sentence)
        if len(chars) < self.q3:
            need_pad = self.q3 - len(chars)
            chars.extend(['<PAD>']*need_pad)
        else:
            chars = chars[:self.q3]
        chars = torch.tensor([self.char2idx[x] for x in chars]).to(torch.int64)
        return chars
    
    def find_max_len(self):
        return max(len(self.make_token(item)) for item in self.sentences)
    
    def find_max_idx(self):
        return self.sentences[np.argmax([len(self.make_token(item)) for item in self.sentences])]

    
    def get_q3(self):
        values = np.array([len(self.make_token(x)) for x in self.sentences])
        return int(np.quantile(values, 1.0))
    
    
    def plot_len(self):
        values = np.array([len(self.make_token(x)) for x in self.sentences])
        plt.hist(values, density=True, bins=80)
        plt.ylabel('count')
        plt.xlabel('length of sequence')
        plt.show()
        print('문장 최대 길이 :',self.max_len)
        results = stats.describe(values)
        print('min={}, max={}, mean={}, Q2={} Q3={}'.format(results[1][0], results[1][1], results[2],
                                                          np.median(values), np.quantile(values, 0.75)))
        

In [9]:
class Net(nn.Module):
    def __init__(self, vocab_size, seq_len, filters, embedding_dim, num_of_kernel):
        super().__init__()
        self.filters=filters
        self.dropout_prob = 0.5
        self.embedding_dim = embedding_dim
        self.num_of_kernel = num_of_kernel
        self.embedding = nn.Embedding(vocab_size, self.embedding_dim)
        self.bn1 = nn.BatchNorm1d(self.num_of_kernel)
        for i in range(len(self.filters)):
            conv = nn.Conv1d(in_channels=self.embedding_dim, out_channels=self.num_of_kernel, kernel_size=self.filters[i])
            setattr(self, f'conv_{i}', conv)
        
        self.dropout = nn.Dropout(p=self.dropout_prob)
        self.bn2 = nn.BatchNorm1d(self.num_of_kernel*len(self.filters))
        self.classifier = nn.Sequential(
            nn.Linear(self.num_of_kernel*len(self.filters), 1),
            nn.Sigmoid()
        )
        
    def get_conv(self, i):
        return getattr(self, f'conv_{i}')
    
    def forward(self, inp):
        x = self.embedding(inp)
        x = x.permute(0, 2, 1) ### embedding 을 transpose해줘야함.안하면 1d-conv이 seq 방향이 아닌, 임베딩 방향으로 진행됨.
        conv_results = [
            F.relu(self.bn1(self.get_conv(i)(x))).permute(0,2,1).max(1)[0]
        for i in range(len(self.filters))]
        x = torch.cat(conv_results, 1)
        x = self.classifier(x)
        x = x.squeeze()
        
        return x

In [10]:
import numpy as np
import random

epochs = 30
batch_size = 128
learning_rates = [4e-5]
filters = [[4,5,6]]
embedding_dim = [1000]
num_of_kernel = [100]
mode = ['char'] # jamo : 자음,모음 단위로 토큰화. char : 한글자 단위로 토큰화

torch.manual_seed(21)
np.random.seed(21)
random.seed(21)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device('cuda')

dataset_train = nlp.data.TSVDataset('./data/train_hate_dataset_v2.txt')
dataset_test = nlp.data.TSVDataset('./data/test_hate_dataset_v2.txt')

best_f1 = 0
for mode0 in mode:
    for learning_rate in learning_rates:
        for embedding_dim0 in embedding_dim:
            for filters0 in filters:
                for num_of_kernel0 in num_of_kernel:
                    
                    print('================')
                    print('mode : {}'.format(mode0))
                    print('lr : {}'.format(learning_rate))
                    print('embedding_dim : {}'.format(embedding_dim0))
                    print('filters : {}'.format(filters0))
                    print('num_of_kernel : {}'.format(num_of_kernel0))
                    print('================')
                    
                    data_train = CharDataset(dataset_train, 0, 1, mode=mode0)
                    data_test = CharDataset(dataset_test, 0, 1, mode=mode0)

                    train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, drop_last=True)
                    test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, drop_last=True)

                    model = Net(data_train.vocab_size, data_train.get_q3(), filters0, embedding_dim0, num_of_kernel0)
                    model.to(device)
                    criterion = nn.BCELoss()
                    #criterion = nn.CrossEntropyLoss()
                    #optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)
                    #optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
                    #optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
                    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.1)

#                     scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
#                                                     lr_lambda=lambda epoch: 0.95 ** epoch,
#                                                     last_epoch=-1,
#                                                     verbose=False)

                    for epoch in range(epochs):

                        running_loss = 0.0
                        correct = 0
                        y_true, y_pred = [], []
                        model.train()
                        for i, data in enumerate(train_dataloader, 0):
                            inputs, labels = data
                            inputs = inputs.to(device)
                            labels = labels.to(device)
                            optimizer.zero_grad()

                            outputs = model(inputs)
                            loss = criterion(outputs, labels)
                            #print('losses :', loss)
                            running_loss = loss.item()
                            loss.backward()
                            optimizer.step()
                            #scheduler.step()
                            pred = (outputs>0.5).to(torch.float)
                            y_pred.extend(pred)
                            y_true.extend(labels)
                        y_true_cpu = [int(x) for x in y_true]
                        y_pred_cpu = [int(x) for x in y_pred]
                        correct = sum([(x==y) for x,y in zip(y_pred, y_true)])
                        precision = precision_score(y_true_cpu, y_pred_cpu)
                        recall = recall_score(y_true_cpu, y_pred_cpu)
                        f1= f1_score(y_true_cpu, y_pred_cpu)
                        print("epoch {} train accuracy {:.3f} f1_score {:.3f} precision {:.3f} recall {:.3f}".format(epoch+1, correct / (len(y_pred)), f1, precision, recall))

                        model.eval()
                        y_true, y_pred = [], []
                        running_loss = 0.0
                        correct = 0
                        for i, data in enumerate(test_dataloader, 0):
                            inputs, labels = data
                            inputs = inputs.to(device)
                            labels = labels.to(device)
                            outputs = model(inputs)
                            pred = (outputs>0.5).to(torch.float)
                            loss = criterion(outputs, labels)
                            running_loss += loss.item()
                            y_pred.extend(pred)
                            y_true.extend(labels)
                        y_true_cpu = [x.cpu() for x in y_true]
                        y_pred_cpu = [x.cpu() for x in y_pred]
                        correct = sum([(x==y) for x,y in zip(y_pred, y_true)])
                        precision = precision_score(y_true_cpu, y_pred_cpu)
                        recall = recall_score(y_true_cpu, y_pred_cpu)
                        f1 = f1_score(y_true_cpu, y_pred_cpu)
                        print("epoch {} test accuracy {:.3f} f1_score {:.3f} precision {:.3f} recall {:.3f}".format(epoch+1, correct / (len(y_pred)), f1, precision, recall))
                        if epoch == 10 and f1 < 0.3:
                            break
                            
                        if f1 > best_f1:
                            best_model = model
                            best_f1 = f1
                    print('Finished Training')

mode : char
lr : 4e-05
embedding_dim : 1000
filters : [4, 5, 6]
num_of_kernel : 100
epoch 1 train accuracy 0.702 f1_score 0.483 precision 0.626 recall 0.393


/home/yeonsik/miniconda3/envs/nlp/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epoch 1 test accuracy 0.633 f1_score 0.000 precision 0.000 recall 0.000
epoch 2 train accuracy 0.771 f1_score 0.615 precision 0.759 recall 0.518


/home/yeonsik/miniconda3/envs/nlp/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epoch 2 test accuracy 0.633 f1_score 0.000 precision 0.000 recall 0.000
epoch 3 train accuracy 0.801 f1_score 0.675 precision 0.799 recall 0.584
epoch 3 test accuracy 0.598 f1_score 0.381 precision 0.438 recall 0.338
epoch 4 train accuracy 0.816 f1_score 0.707 precision 0.808 recall 0.628
epoch 4 test accuracy 0.642 f1_score 0.175 precision 0.565 recall 0.104
epoch 5 train accuracy 0.840 f1_score 0.751 precision 0.840 recall 0.679
epoch 5 test accuracy 0.602 f1_score 0.414 precision 0.450 recall 0.383
epoch 6 train accuracy 0.849 f1_score 0.763 precision 0.857 recall 0.688
epoch 6 test accuracy 0.426 f1_score 0.503 precision 0.369 recall 0.793
epoch 7 train accuracy 0.851 f1_score 0.773 precision 0.840 recall 0.715
epoch 7 test accuracy 0.405 f1_score 0.535 precision 0.375 recall 0.931
epoch 8 train accuracy 0.863 f1_score 0.791 precision 0.861 recall 0.730
epoch 8 test accuracy 0.425 f1_score 0.538 precision 0.382 recall 0.912
epoch 9 train accuracy 0.876 f1_score 0.811 precision 0.87

## save 1dcnn_sentencepiece.pt

In [119]:
torch.save(best_model.state_dict(), './check_point/1dcnn_sentencepiece.pt')

In [118]:
best_f1

0.5424

In [120]:
model = Net(data_train.vocab_size, data_train.get_q3(), [4,5,6], 1000, 100)
model.load_state_dict(torch.load('./check_point/1dcnn_sentencepiece.pt'))
model.eval()
model.to(device)

Net(
  (embedding): Embedding(6403, 1000)
  (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_0): Conv1d(1000, 100, kernel_size=(4,), stride=(1,))
  (conv_1): Conv1d(1000, 100, kernel_size=(5,), stride=(1,))
  (conv_2): Conv1d(1000, 100, kernel_size=(6,), stride=(1,))
  (dropout): Dropout(p=0.5, inplace=False)
  (bn2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (classifier): Sequential(
    (0): Linear(in_features=300, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [121]:
import numpy as np
import random

prefix = './'
epochs = 30
batch_size = 128
learning_rates = [4e-5]
filters = [[4,5,6]]
embedding_dim = [1000]
num_of_kernel = [100]
mode = ['sentencepiece'] # jamo : 자음,모음 단위로 토큰화. char : 한글자 단위로 토큰화

torch.manual_seed(21)
np.random.seed(21)
random.seed(21)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device('cuda')

dataset_train = nlp.data.TSVDataset('./data/train_hate_dataset_v2.txt')
dataset_test = nlp.data.TSVDataset('./data/test_hate_dataset_v2.txt')
                                   
data_train = CharDataset(dataset_train, 0, 1, mode=mode[0])
data_test = CharDataset(dataset_test, 0, 1, mode=mode[0])

In [123]:
sentence = 'ㅆ1발놈아'
model.eval()
with torch.no_grad():
    korean = re.compile('[^1!ㄱ-ㅣ가-힣]+')
    chars = korean.sub('', sentence)
    chars = data_train.make_token(chars)
    if len(chars) < data_train.get_q3():
        need_pad = data_train.get_q3() - len(chars)
        chars.extend(['<PAD>']*need_pad)
    else:
        chars = chars[:data_train.q3]
    chars = torch.tensor([data_train.char2idx[x] for x in chars]).to(torch.int64)
    print(chars)
    chars = chars.to(device)
    chars = torch.unsqueeze(chars, 0)
    prt = [data_train.idx2char[x.item()] for x in chars.squeeze()]
    print(prt)
    print(chars)
    outputs = model(chars)
    print(outputs)

tensor([ 443,  360,   41, 4763, 4205, 5250, 6402, 6402, 6402, 6402, 6402, 6402,
        6402, 6402, 6402, 6402, 6402, 6402, 6402, 6402, 6402, 6402, 6402, 6402,
        6402, 6402, 6402, 6402, 6402, 6402])
['▁', 'ᄊ', '1', '발', '놈', '아', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
tensor([[ 443,  360,   41, 4763, 4205, 5250, 6402, 6402, 6402, 6402, 6402, 6402,
         6402, 6402, 6402, 6402, 6402, 6402, 6402, 6402, 6402, 6402, 6402, 6402,
         6402, 6402, 6402, 6402, 6402, 6402]], device='cuda:0')
tensor(0.8719, device='cuda:0')


## save 1dcnn_jamo.pt

In [103]:
best_model

Net(
  (embedding): Embedding(54, 1000)
  (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_0): Conv1d(1000, 100, kernel_size=(4,), stride=(1,))
  (conv_1): Conv1d(1000, 100, kernel_size=(5,), stride=(1,))
  (conv_2): Conv1d(1000, 100, kernel_size=(6,), stride=(1,))
  (dropout): Dropout(p=0.5, inplace=False)
  (bn2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (classifier): Sequential(
    (0): Linear(in_features=300, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [104]:
best_f1

0.5510373443983403

In [105]:
torch.save(best_model.state_dict(), './check_point/1dcnn_jamo.pt')

In [106]:
import numpy as np
import random

prefix = './'
epochs = 30
batch_size = 128
learning_rates = [4e-5]
filters = [[4,5,6]]
embedding_dim = [1000]
num_of_kernel = [100]
mode = ['jamo'] # jamo : 자음,모음 단위로 토큰화. char : 한글자 단위로 토큰화

torch.manual_seed(21)
np.random.seed(21)
random.seed(21)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device('cuda')

dataset_train = nlp.data.TSVDataset('./data/train_hate_dataset_v2.txt')
dataset_test = nlp.data.TSVDataset('./data/test_hate_dataset_v2.txt')
                                   
data_train = CharDataset(dataset_train, 0, 1, mode='jamo')
data_test = CharDataset(dataset_test, 0, 1, mode='jamo')

In [107]:
model = Net(data_train.vocab_size, data_train.get_q3(), [4,5,6], 1000, 100)
model.load_state_dict(torch.load('./check_point/1dcnn_jamo.pt'))
model.eval()
model.to(device)

Net(
  (embedding): Embedding(54, 1000)
  (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_0): Conv1d(1000, 100, kernel_size=(4,), stride=(1,))
  (conv_1): Conv1d(1000, 100, kernel_size=(5,), stride=(1,))
  (conv_2): Conv1d(1000, 100, kernel_size=(6,), stride=(1,))
  (dropout): Dropout(p=0.5, inplace=False)
  (bn2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (classifier): Sequential(
    (0): Linear(in_features=300, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [116]:
sentence = ''
model.eval()
with torch.no_grad():
    korean = re.compile('[^1!ㄱ-ㅣ가-힣]+')
    chars = korean.sub('', sentence)
    chars = data_train.make_token(chars)
    if len(chars) < data_train.get_q3():
        need_pad = data_train.get_q3() - len(chars)
        chars.extend(['<PAD>']*need_pad)
    else:
        chars = chars[:data_train.q3]
    chars = torch.tensor([data_train.char2idx[x] for x in chars]).to(torch.int64)
    print(chars)
    chars = chars.to(device)
    chars = torch.unsqueeze(chars, 0)
    prt = [data_train.idx2char[x.item()] for x in chars.squeeze()]
    print(prt)
    print(chars)
    outputs = model(chars)
    print(outputs)

tensor([18, 37, 23, 21, 51,  5, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53,
        53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53,
        53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53,
        53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53,
        53, 53, 53, 53, 53, 53, 53, 53, 53])
['ㅂ', 'ㅕ', 'ㅇ', 'ㅅ', 'ㅣ', 'ㄴ', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '

## save 1dcnn_char.pt

In [125]:
best_model

Net(
  (embedding): Embedding(1668, 1000)
  (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_0): Conv1d(1000, 100, kernel_size=(4,), stride=(1,))
  (conv_1): Conv1d(1000, 100, kernel_size=(5,), stride=(1,))
  (conv_2): Conv1d(1000, 100, kernel_size=(6,), stride=(1,))
  (dropout): Dropout(p=0.5, inplace=False)
  (bn2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (classifier): Sequential(
    (0): Linear(in_features=300, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [126]:
best_f1

0.5409356725146198

In [127]:
torch.save(best_model.state_dict(), './check_point/1dcnn_char.pt')

In [128]:
import numpy as np
import random

prefix = './'
epochs = 30
batch_size = 128
learning_rates = [4e-5]
filters = [[4,5,6]]
embedding_dim = [1000]
num_of_kernel = [100]
mode = ['char'] # jamo : 자음,모음 단위로 토큰화. char : 한글자 단위로 토큰화

torch.manual_seed(21)
np.random.seed(21)
random.seed(21)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device('cuda')

dataset_train = nlp.data.TSVDataset('./data/train_hate_dataset_v2.txt')
dataset_test = nlp.data.TSVDataset('./data/test_hate_dataset_v2.txt')
                                   
data_train = CharDataset(dataset_train, 0, 1, mode='char')
data_test = CharDataset(dataset_test, 0, 1, mode='char')

In [129]:
model = Net(data_train.vocab_size, data_train.get_q3(), [4,5,6], 1000, 100)
model.load_state_dict(torch.load('./check_point/1dcnn_char.pt'))
model.eval()
model.to(device)

Net(
  (embedding): Embedding(1668, 1000)
  (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_0): Conv1d(1000, 100, kernel_size=(4,), stride=(1,))
  (conv_1): Conv1d(1000, 100, kernel_size=(5,), stride=(1,))
  (conv_2): Conv1d(1000, 100, kernel_size=(6,), stride=(1,))
  (dropout): Dropout(p=0.5, inplace=False)
  (bn2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (classifier): Sequential(
    (0): Linear(in_features=300, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [7]:
model = best_model
sentence = '돼지같은놈 ㅋㅋ'
model.eval()
with torch.no_grad():
    korean = re.compile('[^1!ㄱ-ㅣ가-힣]+')
    chars = korean.sub('', sentence)
    chars = data_train.make_token(chars)
    if len(chars) < data_train.get_q3():
        need_pad = data_train.get_q3() - len(chars)
        chars.extend(['<PAD>']*need_pad)
    else:
        chars = chars[:data_train.q3]
    chars = torch.tensor([data_train.char2idx[x] for x in chars]).to(torch.int64)
    print(chars)
    chars = chars.to(device)
    chars = torch.unsqueeze(chars, 0)
    prt = [data_train.idx2char[x.item()] for x in chars.squeeze()]
    print(prt)
    print(chars)
    outputs = model(chars)
    print(outputs)

tensor([ 387, 1223,   46, 1106,  291,   17,   17, 1667, 1667, 1667, 1667, 1667,
        1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667,
        1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667])
['돼', '지', '같', '은', '놈', 'ㅋ', 'ㅋ', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
tensor([[ 387, 1223,   46, 1106,  291,   17,   17, 1667, 1667, 1667, 1667, 1667,
         1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667,
         1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667, 1667]],
       device='cuda:0')
tensor(0.5369, device='cuda:0')
